## import

In [12]:
import time
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import re #정규표현식
from PIL import Image
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests

def download(url, file_name):
    with open(file_name, "wb") as file:   # open in binary mode
        response = requests.get(url)       # get request
        file.write(response.content)      # write to fill

## test code

In [4]:
# 드라이버 실행
driver = webdriver.Chrome()  # 빈칸 시 path 검색(optional argument)
driver.get('https://esg.krx.co.kr/contents/02/02030000/ESG02030000.jsp')
time.sleep(1) 
next_page = driver.find_element(By.CSS_SELECTOR, "ul > li.next > a")
page_num = 1

In [5]:
# 2021년으로 이동
driver.find_element(By.CSS_SELECTOR, "#sch_yyc4ca4238a0b923820dcc509a6f75849b").click()
time.sleep(0.2)
driver.find_element(By.CSS_SELECTOR, "#sch_yyc4ca4238a0b923820dcc509a6f75849b > option:nth-child(2)").click()
time.sleep(0.2)
driver.find_element(By.CSS_SELECTOR, "#btnidc4ca4238a0b923820dcc509a6f75849b").click()

In [6]:
# 회사 클릭
company = driver.find_element(By.CSS_SELECTOR, "tbody > tr:nth-child({}) > td:nth-child(1)".format(1)).get_attribute('innerHTML') # 회사명 저장
ESG = driver.find_element(By.CSS_SELECTOR, "tbody > tr:nth-child({}) > td:nth-child(6) > a.file-download.useK".format(1)).click() # i번째 회사 클릭
time.sleep(0.3)

In [7]:
# 보고서 프레임으로 이동
driver.switch_to.window(driver.window_handles[-1]) # 새 탭으로 전환
driver.find_element(By.CSS_SELECTOR, "#attachedDoc").click() # 목록 열기
time.sleep(0.3)
driver.find_element(By.CSS_SELECTOR, "#attachedDoc > option:nth-child(2)").click() # 목록에서 2번째
time.sleep(0.3)
driver.switch_to.frame("docViewFrm")

In [8]:
# 보고서 링크 클릭
time.sleep(0.3)
Report = driver.find_element(By.CSS_SELECTOR, "body > div > div > a") # 보고서 클릭
Report.click()
driver.switch_to.window(driver.window_handles[-1]) # 보고서로 탭 전환
URL = driver.current_url # 현재 링크 저장

In [9]:
# 다운로드
download(URL, "ESG_Reports_File/{}.pdf".format(company)) # 회사명으로 파일 저장
driver.close()
driver.switch_to.window(driver.window_handles[-1])
driver.close()

## Main Code

### 일반 코드

In [1]:
#경로 세팅
os.mkdir("ESG_Reports_File_22") # 한 번 만 실행

In [30]:
path = "ESG_Reports_File_22"
res = os.listdir(path) # 저장 경로

# 크롤러 실행
driver = webdriver.Chrome()  # 빈칸 시 path 검색(optional argument)
driver.get('https://esg.krx.co.kr/contents/02/02030000/ESG02030000.jsp')
time.sleep(2)

# 초기 변수
page_num = 1
error_count = 1
same = 0
eng_report_list = []

# 2021년으로 이동
# driver.find_element(By.CSS_SELECTOR, "#sch_yyc4ca4238a0b923820dcc509a6f75849b").click()
# time.sleep(0.2)
# driver.find_element(By.CSS_SELECTOR, "#sch_yyc4ca4238a0b923820dcc509a6f75849b > option:nth-child(2)").click()
# time.sleep(0.2)
# driver.find_element(By.CSS_SELECTOR, "#btnidc4ca4238a0b923820dcc509a6f75849b").click()

try:
    while True:
        print(page_num, "번째 페이지 탐색중 ...")
        for i in range(1, 11):
            # 회사 탐색        
            company = driver.find_element(By.CSS_SELECTOR, "tbody > tr:nth-child({}) > td:nth-child(1)".format(i)).get_attribute('innerHTML') # 회사명 저장
            print(i, '번째 회사 탐색 :', company, end=" | ")

            for j in range(len(res)):
                if (company in res[j][:-4]) == True:
                    same = 1
                    break
                
            if same == 1:
                print(company, "는(은) 이미 존재하는 보고서입니다.")
                same = 0
                continue

            # 회사 클릭
            driver.find_element(By.CSS_SELECTOR, "tbody > tr:nth-child({}) > td:nth-child(6) > a.file-download.useK".format(i)).click() # i번째 회사 클릭
            time.sleep(0.6)

            # 보고서 프레임으로 이동
            try:
                driver.switch_to.window(driver.window_handles[-1]) # 새 탭으로 전환
                driver.find_element(By.CSS_SELECTOR, "#attachedDoc").click() # 목록 열기
                time.sleep(0.9)
                driver.find_element(By.CSS_SELECTOR, "#attachedDoc > option:nth-child(2)").click() # 목록에서 2번째
                time.sleep(0.6)
                driver.switch_to.frame("docViewFrm")
                
            except: # 간혹 실패한 경우
                if error_count < 3:
                    print("탐색 중 오류가 발생해 다시 시도합니다. ", error_count, "회")
                    driver.switch_to.window(driver.window_handles[-1])
                    driver.close()
                    i -= 1
                    driver.switch_to.window(driver.window_handles[-1])
                    error_count += 1
                    continue
            
            # 보고서 링크 클릭
            time.sleep(0.6)

            try: # 영문임을 가정하고 실행
                time.sleep(0.2)
                Report_kor = driver.find_element(By.CSS_SELECTOR, "body > div > div > a:nth-child(4)") # 보고서 클릭
                Report_eng = driver.find_element(By.CSS_SELECTOR, "body > div > div > a:nth-child(5)")
                print(company + " 영문 PDF 탐지", end=" -> ")
                eng_report_list.append(company)
                
                Report_kor.click()
                Report_eng.click()
                time.sleep(1)

                driver.switch_to.window(driver.window_handles[-1]) # 보고서로 탭 전환
                URL_eng = driver.current_url # 현재 링크 저장
                driver.close() # 목적 완수 후 보고서 페이지 닫기
                time.sleep(0.2)

                driver.switch_to.window(driver.window_handles[-1])
                URL_kor = driver.current_url # 현재 링크 저장
                driver.close() # 목적 완수 후 보고서 페이지 닫기
                time.sleep(0.2)

                driver.switch_to.window(driver.window_handles[-1]) # 보고서로 탭 전환
                driver.close() # 공시 페이지 닫기
                time.sleep(0.2)

                print(company + " 영문, 국문 보고서 다운로드")
                download(URL_kor, path + "/{}_1.pdf".format(company)) # 회사명으로 파일 저장
                download(URL_eng, path + "/{}_2.pdf".format(company)) # 회사명으로 파일 저장
                
            except: # 국문 보고서 시도하기
                time.sleep(0.2)
                Report = driver.find_element(By.CSS_SELECTOR, "body > div > div > a").click() # 보고서 클릭
                driver.switch_to.window(driver.window_handles[-1]) # 보고서로 탭 전환
                URL = driver.current_url # 현재 링크 저장
                
                # 목적 완수 후 보고서 페이지 닫기
                driver.close()
                time.sleep(0.3)
                driver.switch_to.window(driver.window_handles[-1])
                
                driver.close() # 공시 페이지 닫기
                time.sleep(0.2)

                print(company + " 국문 보고서 다운로드")
                download(URL, path + "/{}.pdf".format(company)) # 회사명으로 파일 저장

            driver.switch_to.window(driver.window_handles[-1]) # 다시 KRX 페이지로 전환
            
        driver.find_element(By.CSS_SELECTOR, "ul > li.next > a").click() # 다음 페이지로 이동하기
        time.sleep(1.5)
        page_num += 1
except:
    print("더 이상 읽을 페이지가 없습니다.")
    print("총 탐색한 기업 수 :", ((page_num - 1) * 10) + i - 1)

1 번째 페이지 탐색중 ...
1 번째 회사 탐색 : 넥센타이어 | 넥센타이어 는(은) 이미 존재하는 보고서입니다.
2 번째 회사 탐색 : 두산밥캣 | 두산밥캣 는(은) 이미 존재하는 보고서입니다.
3 번째 회사 탐색 : LG | LG 는(은) 이미 존재하는 보고서입니다.
4 번째 회사 탐색 : 한국가스공사 | 한국가스공사 는(은) 이미 존재하는 보고서입니다.
5 번째 회사 탐색 : 삼양사 | 삼양사 는(은) 이미 존재하는 보고서입니다.
6 번째 회사 탐색 : 삼양홀딩스 | 삼양홀딩스 는(은) 이미 존재하는 보고서입니다.
7 번째 회사 탐색 : 미래에셋생명 | 미래에셋생명 는(은) 이미 존재하는 보고서입니다.
8 번째 회사 탐색 : 쌍용씨앤이 | 쌍용씨앤이 는(은) 이미 존재하는 보고서입니다.
9 번째 회사 탐색 : 강원랜드 | 강원랜드 는(은) 이미 존재하는 보고서입니다.
10 번째 회사 탐색 : HL홀딩스 | HL홀딩스 는(은) 이미 존재하는 보고서입니다.
2 번째 페이지 탐색중 ...
1 번째 회사 탐색 : 삼성증권 | 삼성증권 는(은) 이미 존재하는 보고서입니다.
2 번째 회사 탐색 : 현대글로비스 | 현대글로비스 는(은) 이미 존재하는 보고서입니다.
3 번째 회사 탐색 : KC그린홀딩스 | KC그린홀딩스 는(은) 이미 존재하는 보고서입니다.
4 번째 회사 탐색 : 한화에어로스페이스 | 한화에어로스페이스 는(은) 이미 존재하는 보고서입니다.
5 번째 회사 탐색 : SK | SK 는(은) 이미 존재하는 보고서입니다.
6 번째 회사 탐색 : 한일시멘트 | 한일시멘트 는(은) 이미 존재하는 보고서입니다.
7 번째 회사 탐색 : 현대건설 | 현대건설 는(은) 이미 존재하는 보고서입니다.
8 번째 회사 탐색 : 케이티앤지 | 케이티앤지 는(은) 이미 존재하는 보고서입니다.
9 번째 회사 탐색 : LG생활건강 | LG생활건강 는(은) 이미 존재하는 보고서입니다.
10 번째 회사 탐색 : 한샘 | 한샘 는(은) 이미 존재하는 보고서입니다.
3 번째 페이지 탐색중 .

### 영문 우선순위로 탐색

In [ ]:
#경로 세팅
os.mkdir("ESG_ENG_Reports_File") # 한 번 만 실행

In [ ]:
path = "ESG_Reports_File_22"
res = os.listdir(path) # 저장 경로

# 크롤러 실행
driver = webdriver.Chrome()  # 빈칸 시 path 검색(optional argument)
driver.get('https://esg.krx.co.kr/contents/02/02030000/ESG02030000.jsp')
time.sleep(2)

# 초기 변수
page_num = 1
error_count = 1
same = 0
eng_report_list = []

# 2021년으로 이동
# driver.find_element(By.CSS_SELECTOR, "#sch_yyc4ca4238a0b923820dcc509a6f75849b").click()
# time.sleep(0.2)
# driver.find_element(By.CSS_SELECTOR, "#sch_yyc4ca4238a0b923820dcc509a6f75849b > option:nth-child(2)").click()
# time.sleep(0.2)
# driver.find_element(By.CSS_SELECTOR, "#btnidc4ca4238a0b923820dcc509a6f75849b").click()

try:
    while True:
        print(page_num, "번째 페이지 탐색중 ...")
        for i in range(1, 11):
            # 회사 탐색        
            company = driver.find_element(By.CSS_SELECTOR, "tbody > tr:nth-child({}) > td:nth-child(1)".format(i)).get_attribute('innerHTML') # 회사명 저장
            print(i, '번째 회사 탐색 :', company, end=" | ")

            for j in range(len(res)):
                if (company in res[j][:-4]) == True:
                    same = 1
                    break
                
            if same == 1:
                print(company, "는(은) 이미 존재하는 보고서입니다.")
                same = 0
                continue

            # 회사 클릭
            driver.find_element(By.CSS_SELECTOR, "tbody > tr:nth-child({}) > td:nth-child(6) > a.file-download.useK".format(i)).click() # i번째 회사 클릭
            time.sleep(0.6)

            # 보고서 프레임으로 이동
            try:
                driver.switch_to.window(driver.window_handles[-1]) # 새 탭으로 전환
                driver.find_element(By.CSS_SELECTOR, "#attachedDoc").click() # 목록 열기
                time.sleep(0.9)
                driver.find_element(By.CSS_SELECTOR, "#attachedDoc > option:nth-child(2)").click() # 목록에서 2번째
                time.sleep(0.6)
                driver.switch_to.frame("docViewFrm")
                
            except: # 간혹 실패한 경우
                if error_count < 3:
                    print("탐색 중 오류가 발생해 다시 시도합니다. ", error_count, "회")
                    driver.switch_to.window(driver.window_handles[-1])
                    driver.close()
                    i -= 1
                    driver.switch_to.window(driver.window_handles[-1])
                    error_count += 1
                    continue
            
            # 보고서 링크 클릭
            time.sleep(0.6)

            try: # 영문임을 가정하고 실행
                time.sleep(0.2)
                Report_kor = driver.find_element(By.CSS_SELECTOR, "body > div > div > a:nth-child(4)") # 보고서 클릭
                Report_eng = driver.find_element(By.CSS_SELECTOR, "body > div > div > a:nth-child(5)")
                print(company + " 영문 PDF 탐지", end=" -> ")
                eng_report_list.append(company)
                
                Report_kor.click()
                Report_eng.click()
                time.sleep(1)

                driver.switch_to.window(driver.window_handles[-1]) # 보고서로 탭 전환
                URL_eng = driver.current_url # 현재 링크 저장
                driver.close() # 목적 완수 후 보고서 페이지 닫기
                time.sleep(0.2)

                driver.switch_to.window(driver.window_handles[-1])
                URL_kor = driver.current_url # 현재 링크 저장
                driver.close() # 목적 완수 후 보고서 페이지 닫기
                time.sleep(0.2)

                driver.switch_to.window(driver.window_handles[-1]) # 보고서로 탭 전환
                driver.close() # 공시 페이지 닫기
                time.sleep(0.2)

                print(company + " 영문, 국문 보고서 다운로드")
                download(URL_kor, path + "/{}_1.pdf".format(company)) # 회사명으로 파일 저장
                download(URL_eng, path + "/{}_2.pdf".format(company)) # 회사명으로 파일 저장
                
            except: # 국문 보고서 시도하기
                print(company, "는 국문 보고서만 존재합니다. 패스합니다.")
                driver.close() # 공시 페이지 닫기
                time.sleep(0.2)
                pass

            driver.switch_to.window(driver.window_handles[-1]) # 다시 KRX 페이지로 전환
            
        driver.find_element(By.CSS_SELECTOR, "ul > li.next > a").click() # 다음 페이지로 이동하기
        time.sleep(1.5)
        page_num += 1
except:
    print("더 이상 읽을 페이지가 없습니다.")
    print("총 탐색한 기업 수 :", ((page_num - 1) * 10) + i - 1)